# Team.Cayley 2팀 집단감염 분석

Team members: 박하람, 김민석, 김영규, 신경민, 최은혜, 신주영, 설지은

👀 What we've done for a month! 
1. 정례브리핑 집단감염 자료 데이터화: plain text 👉 [google sheets](https://docs.google.com/spreadsheets/d/1PtEuhh5rPJKlCsgdAp3uDr8tb_-v1yZLSKn49ReAbzQ/edit#gid=1472469111)
2. 집단감염 경로 분류작업: for 3 and half an hour...🥲
3. 데이터 분석 with Plotly
  - 월별 집단감염 건수 시각화
  - 분류기준별 집단감염 건수 시각화
  - 분류기준별 집단감염 확진자수 시각화
  - 지역별 집단감염 건수 시각화
  - 지역별 집단감염 확진자수 시각화

##  1. 데이터 불러오기

In [39]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import datetime
from datetime import date, timedelta
import warnings
warnings.filterwarnings(action='ignore')

In [40]:
df = pd.read_csv('집단감염 데이터.csv', index_col=[0])
df = df.sort_values('caseId')
df = df.reset_index()
df.head(12)

,regionId,province,city,date,caseId,infection_case,confirmed,accumulated,briefingdate,classname,note
0,10,서울,은평구,2020-02-23,1,은평성모병원,2.0,2.0,2020-02-23,의료기관,NaN
1,10,서울,은평구,2020-02-23,1,은평성모병원,9.0,11.0,2020-02-27,의료기관,NaN
2,10,서울,은평구,2020-02-23,1,은평성모병원,3.0,14.0,2020-03-03,의료기관,NaN
3,10,서울,종로구,2020-03-03,2,명륜교회,3.0,3.0,2020-03-03,종교시설 및 단체,NaN
4,10,서울,종로구,2020-03-03,3,종로노인종합복지관,6.0,6.0,2020-03-03,복지시설 및 단체,NaN
5,10,서울,성동구,2020-03-04,4,성동구 아파트,12.0,12.0,2020-03-04,거주지,NaN
6,10,서울,성동구,2020-03-04,4,성동구 아파트,1.0,13.0,2020-03-05,거주지,NaN
7,10,서울,구로구,2020-03-10,5,콜센터,2.0,158.0,2020-03-24,콜센터,NaN
8,10,서울,구로구,2020-03-10,5,콜센터,4.0,156.0,2020-03-23,콜센터,NaN
9,10,서울,구로구,2020-03-10,5,콜센터,4.0,152.0,2020-03-22,콜센터,NaN


## 2. 집단감염 건수 월별통계

###전처리

In [41]:
## 중복 사건 중 최신의 사건만 사용.
class_data = df.drop_duplicates('caseId',keep='last')
class_data = class_data.reset_index(drop=True)
column = ['date', 'classname']
cls_date1 = class_data[column]
cls_date1

,date,classname
0,2020-02-23,의료기관
1,2020-03-03,종교시설 및 단체
2,2020-03-03,복지시설 및 단체
3,2020-03-04,거주지
4,2020-03-10,콜센터
...,...,...
986,2021-02-25,"체육, 여가시설"
987,2021-02-24,"가족, 지인 및 소모임"
988,2021-02-19,"가족, 지인 및 소모임"
989,2021-02-22,"가족, 지인 및 소모임"


In [42]:
## 사건발생 날짜를 월 단위로 변경.
cls_date2 = cls_date1.dropna(axis=0)
date_list = cls_date2['date'].to_list()

month_list = []
for i in date_list:
    if i[:4] == '2021':
        month = i[5:7]
        month_list.append('21년 ' + month + '월')
    else:
        month = i[5:7]
        month_list.append('20년 ' + month + '월')

## 월별, 분류별 발생 변환
cls_date2['date'] = month_list
cls_date2.columns = ['month', 'name']
cls_date2 = cls_date2.sort_values(by=['month'],ascending=True)

cls_count = cls_date1.value_counts('classname')
cls_count = pd.DataFrame(cls_count).reset_index()
cls_count.columns = ['name', 'count']
cls_count = cls_count.sort_values(by='count', ascending=True)

In [43]:
## 월별 집단감염 발생 통계
cls_plot = cls_date2.value_counts('month')
cls_plot = pd.DataFrame(cls_plot)
cls_plot = cls_plot.reset_index().sort_values(by='month')
cls_plot.columns = ['month', 'count']

fig = px.bar(cls_plot, x='month', y='count', text='count')
fig.update_layout(title={'text': "<b>월별 집단감염 건수(누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='누적확진자수',tickfont_size=12))

fig.show()

## 3. 집단감염 집단분류별 사건 발생 수

---
집단감염이 발생한 위치적 특징을 유사한 기준끼리 분류하여 나타낸 통계로, 분류별 사건 발생 수를 집중적으로 살펴본다.

1. 데이터를 'classname(분류)' 컬럼 별로 시각화 한 결과.
2. 같은 사건에 대해서는 가장 최근 브리핑의 사건만을 이용하여 중복을 삭제함.
3. 'Animated Visualization'은, '분류' 컬럼 별 통계를 월별로, 월별 누적으로 나타냄.
4. 월별 집단발생 수 시각화 중, 사건 발생 날짜가 명시되지 않은 경우는 그 데이터를 삭제함.

###데이터 시각화

In [44]:
## 분류별 집단감염 발생 통계
fig = px.scatter(cls_count, x='name', y='count', size='count', color='name', size_max=50, title='분류별 집단감염 발생 수', text='count')
fig.show()

### Animated Visualization

#### 분류별 사건 발생(월별)

In [45]:
date_count = cls_date2.value_counts()
date_count = pd.DataFrame(date_count)
date_count = date_count.sort_values(by=['month'])

date_count = date_count.reset_index()
date_count.columns = ['month', 'name', 'count']

## 특정 분류의 사건이 발생하지 않은 월을 0으로 추가.
date_pivot = pd.pivot_table(date_count,index='month',columns='name', values='count', fill_value=0)
date_pivot = date_pivot.unstack().rename('count').reset_index()
date_pivot

,name,month,count
0,"가족, 지인 및 소모임",20년 02월,0
1,"가족, 지인 및 소모임",20년 03월,1
2,"가족, 지인 및 소모임",20년 04월,0
3,"가족, 지인 및 소모임",20년 05월,1
4,"가족, 지인 및 소모임",20년 06월,8
...,...,...,...
359,회사,20년 10월,0
360,회사,20년 11월,1
361,회사,20년 12월,2
362,회사,21년 01월,0


In [46]:
## 분류별 집단감염 발생 수의 월별 변화.
anime = px.bar(date_pivot, x="name", y="count", animation_frame="month", animation_group="name",
           range_y=[0,60], color="name", range_color=[0,60], hover_name="name")
anime.update_layout(title={'text': "<b>분류기준별 집단감염 건수(월별)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                 title_font=dict(size=19,color='black'),
                 xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='확진자수',tickfont_size=12))

anime.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 900
anime.show()

#### 분류별 사건 발생(누적)

In [47]:
acc_pivot = date_pivot.copy()

## count를 누적으로 나타내기
name={}
for i in range(len(acc_pivot)):  
  if i == len(acc_pivot)-1:
    name[acc_pivot['name'][i]] =  acc_pivot['count'][i]
  elif acc_pivot['name'][i] == acc_pivot['name'][i+1]:
    acc_pivot['count'][i+1] = acc_pivot['count'][i] + acc_pivot['count'][i+1]
  else:
    name[acc_pivot['name'][i]] =  acc_pivot['count'][i]

abc = pd.DataFrame(name, index=[0])
abc = abc.transpose().reset_index()
abc.columns = ['name', 'count']
abc = abc.sort_values('count').reset_index(drop=True)
abc


## 누적이 낮은 순서대로
a = acc_pivot.groupby('name')['count'].sum()
b = pd.DataFrame(a).sort_values('count')
ani = pd.DataFrame([('월', '이름', '숫자')], columns = ['month' , 'name', 'count'])

for i in range(len(abc)):
  for j in range(len(acc_pivot)):
    if acc_pivot['name'][j] == abc['name'][i]:
      ani = ani.append(acc_pivot.iloc[j])
    else: continue
ani = ani.iloc[1:]
ani

,month,name,count
299,20년 02월,집회,0
300,20년 03월,집회,0
301,20년 04월,집회,0
302,20년 05월,집회,0
303,20년 06월,집회,0
...,...,...,...
8,20년 10월,"가족, 지인 및 소모임",55
9,20년 11월,"가족, 지인 및 소모임",85
10,20년 12월,"가족, 지인 및 소모임",110
11,21년 01월,"가족, 지인 및 소모임",128


In [48]:
## 분류별 집단감염 발생 수의 누적.
anime = px.bar(ani, x='name', y="count", animation_frame="month", animation_group="name",
           range_y=[0,170], color="name", hover_name="name")
anime.update_layout(title={'text': "<b>분류기준별 집단감염 건수(누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                 title_font=dict(size=20,color='black'),
                 xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='누적확진자수',tickfont_size=12))


anime.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 700
anime.show()

## 3. 집단감염 집단분류별 확진자수

###데이터 전처리

In [49]:
df1=df.copy()
df1.drop(['regionId','note','date','confirmed','city'],axis='columns', inplace=True)
df1 = df1.drop_duplicates('caseId',keep='last')
df2=df1.groupby("classname").sum()
df2.drop(['caseId'],axis='columns', inplace=True)
df3=df2.sort_values(by=['accumulated'], axis=0, ascending=True)
df3

,accumulated
classname,
숙박업,15.0
항구/선박,24.0
장례식장,82.0
회사,85.0
공공기관,133.0
설명회,147.0
거주지,163.0
건설업,180.0
공공장소,192.0


###데이터 시각화

In [50]:
cn=df3.index
ac=df3['accumulated']
fig = go.Figure([go.Bar(x=cn, y=ac, text=ac, textposition='outside')])
fig.update_traces(marker_color='#9238b0', opacity=0.7)
fig.update_layout(title={'text': "<b>분류기준별 집단감염 확진자 수 (누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=20,color='black'),
                  xaxis_tickfont_size=12, yaxis=dict(title='누적확진자수',tickfont_size=14))

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

### Animated Visualization

#### 분류별 확진자 수(월별)

In [51]:
df1=df.copy()
df1.dropna(subset=['briefingdate'],axis=0,inplace=True)
df1.dropna(subset=['classname'],axis=0,inplace=True)
df1.dropna(subset=['accumulated'],axis=0,inplace=True)
df1.dropna(subset=['confirmed'],axis=0,inplace=True)
df1.drop(['regionId','note','date','city','province'],axis='columns', inplace=True)
df1.head(10)

,caseId,infection_case,confirmed,accumulated,briefingdate,classname
0,1,은평성모병원,2.0,2.0,2020-02-23,의료기관
1,1,은평성모병원,9.0,11.0,2020-02-27,의료기관
2,1,은평성모병원,3.0,14.0,2020-03-03,의료기관
3,2,명륜교회,3.0,3.0,2020-03-03,종교시설 및 단체
4,3,종로노인종합복지관,6.0,6.0,2020-03-03,복지시설 및 단체
5,4,성동구 아파트,12.0,12.0,2020-03-04,거주지
6,4,성동구 아파트,1.0,13.0,2020-03-05,거주지
7,5,콜센터,2.0,158.0,2020-03-24,콜센터
8,5,콜센터,4.0,156.0,2020-03-23,콜센터
9,5,콜센터,4.0,152.0,2020-03-22,콜센터


In [52]:
dfmon=df1

#월별 데이터프레임 저장할 동적변수 생성
names = []
for i in range(0,13):
  names.append("df"+str(i+1))

time = ['2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12','2021-01','2021-02']

#데이터프레임 월별 분리 후 변수에 저장
for i,j in zip(names,time):
  globals()[i]=dfmon[dfmon['briefingdate'].str.contains(j)]

import sys , numpy as np
mod = sys.modules[__name__]

#지역별로 묶고 sum한 월별 데이터프레임을 list a에 저장
a=[]
for i in names:
  a.append(getattr(mod,i).groupby('classname').sum())

#a에서 꺼내서 데이터프레임마다 month 컬럼 추가
for i,j in zip(a,time):
  i['month']=j

#데이터 프레임 병합
coromonth= pd.concat([a[0], a[1]],axis=0)
for i in a[2:]:
  coromonth = pd.concat([coromonth,i])

#월별 classname 개수 동일하게 만들기 
coromonth=pd.pivot_table(coromonth,index='month',columns='classname', values='confirmed', fill_value=0)
coromonth=coromonth.unstack().rename('accumulated').reset_index()

In [53]:
fig = px.bar(coromonth, x='classname', y="accumulated", color='classname',animation_frame="month",animation_group='classname',range_y=[0,4000])

fig.update_layout(title={'text': "<b>분류기준별 집단감염 확진자 수 (월별)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='확진자수',tickfont_size=12))

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 900

fig.show()

####분류별 확진자 수(누적)

In [54]:
#데이터프레임 월별 분리 후 변수에 저장
for i,j in zip(names,time):
  globals()[i]=coromonth[coromonth['month']==j]

#각 데이터 프레임 인덱스 reset
for i in names:
  getattr(mod,i).reset_index(drop=True,inplace=True)

#익월=금월+익월로 누적값 생성
for i in range(1,13):
  getattr(mod,'df'+'%d'%(i+1))['accumulated']= getattr(mod,'df'+'%d'%i)['accumulated']+ getattr(mod,'df'+'%d'%(i+1))['accumulated']

#마지막 달의 그래프 내림차순 정렬을 위해 df1 인덱스 재지정
df13.sort_values(by = 'accumulated', ascending = True, inplace=True)
df1=df1.reindex(index =df13.index.tolist())

#데이터프레임 병합
accum = pd.concat([df1,df2],axis=0)
for i in names[2:]:
  accum = pd.concat([accum,getattr(mod,i)])
case_accum=accum

In [55]:
fig = px.bar(case_accum, x='classname', y="accumulated", color='classname',animation_frame="month",animation_group='classname',range_y=[0,11000])

fig.update_layout(title={'text': "<b>분류기준별 집단감염 확진자 수 (누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='누적확진자수',tickfont_size=12))

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 700

fig.show()

##분류별 집단감염 사건 수와 환자 수

In [57]:
ani.rename(columns ={'name':'classname'},inplace=True)

#acc_pivot과 case_accum 월 형식 통일
acc_list=ani['month'].to_list()
month2=[]
for i in acc_list:
  if i[:3]=='20년':
    month=i[4:6]
    month2.append('2020'+'-'+month)
  else:
    month=i[4:6]
    month2.append('2021'+'-'+month)
ani['month']=month2

accum=pd.merge(case_accum, ani, on=['classname','month'], sort=False)
accum.rename(columns = {'count' : '누적사건수','accumulated':'누적확진자수'}, inplace = True)
accum

In [59]:
fig = px.scatter(accum, x="누적사건수", y="누적확진자수", animation_frame="month", animation_group="classname",
                 size="누적확진자수", color="classname", hover_name="classname", size_max=55, range_x=[0,180],range_y=[0,12500])

fig.update_layout(title={'text': "<b>분류기준별 집단감염 건수와 확진자 수 (누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='누적사건수',tickfont_size=12), yaxis=dict(title='누적확진자수',tickfont_size=12))

## 4. 지역별 집단감염 사건 수

####지역별 집단감염 수(월별)

In [60]:
df.dropna(subset=['briefingdate'],axis=0,inplace=True)
df.dropna(subset=['province'],axis=0,inplace=True)
df.dropna(subset=['infection_case'],axis=0,inplace=True)
df.sort_values(by=['briefingdate'],ascending=True,inplace=True)
df

,regionId,province,city,date,caseId,infection_case,confirmed,accumulated,briefingdate,classname,note
2291,60,경상북도,청도군,2020-02-18,757,청도대남병원,1.0,1.0,2020-02-18,요양병원 및 시설,31번 환자 접촉자
1053,12,대구,수성구,2020-02-18,297,신천지 대구교회,1.0,1.0,2020-02-18,종교시설 및 단체,31번 확진자 경로=신천지 대구교회와 청도대남병원
2292,60,경상북도,청도군,2020-02-18,757,청도대남병원,1.0,2.0,2020-02-19,요양병원 및 시설,NaN
1054,12,대구,수성구,2020-02-18,297,신천지 대구교회,14.0,15.0,2020-02-19,종교시설 및 단체,NaN
2294,60,경상북도,청도군,2020-02-18,757,청도대남병원,2.0,4.0,2020-02-20,요양병원 및 시설,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2892,60,경상북도,의성군,2021-02-16,963,가족모임,1.0,88.0,2021-02-28,"가족, 지인 및 소모임",NaN
2907,10,서울,동대문구,2021-02-25,971,아동시설,8.0,9.0,2021-02-28,복지시설 및 단체,NaN
2790,20,경기도,남양주시,2021-02-13,938,플라스틱공장,2.0,191.0,2021-02-28,제조업,NaN
2825,20,경기도,용인시,2021-02-17,944,요양원/어린이집,13.0,48.0,2021-02-28,요양병원 및 시설,NaN


In [61]:
#월별 데이터프레임 저장할 동적변수 생성
names = []
for i in range(0,13):
  names.append("df"+str(i+1))

time = ['2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12','2021-01','2021-02']

#각 케이스의 마지막 확진 데이터만 남김
dfmon=df.drop_duplicates('caseId',keep='last')

import sys , numpy as np
mod = sys.modules[__name__]

#데이터프레임 월별 분리 후 변수에 저장
for i,j in zip(names,time):
  globals()[i]=dfmon[dfmon['briefingdate'].str.contains(j)]

#지역별로 묶고 count한 월별 데이터프레임을 list a에 저장
b=[]
for i in names:
  b.append(globals()[i].groupby(by = 'province').count())

#a에서 꺼내서 데이터프레임마다 month 컬럼 추가
for i,j in zip(b,time):
  i['month']=j

#데이터 프레임 병합
casemonth= pd.concat([b[0], b[1]],axis=0)
for i in b[2:]:
  casemonth = pd.concat([casemonth,i])

#월별 province 개수 동일하게 만들기 
casemonth=pd.pivot_table(casemonth,index='month',columns='province', values='caseId', fill_value=0)
casemonth=casemonth.unstack().rename('caseId').reset_index()

In [62]:
fig = px.bar(casemonth, x='province', y="caseId", color='province',animation_frame="month",animation_group='province',range_y=[0,55])

fig.update_layout(title={'text': "<b>지역별 집단감염 건수(월별)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='확진자수',tickfont_size=12))

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 900
fig.show()

####지역별 집단감염 수(누적)

In [63]:
#데이터프레임 월별 분리 후 변수에 저장
for i,j in zip(names,time):
  globals()[i]=casemonth[casemonth['month']==j]

#각 데이터 프레임 인덱스 reset
for i in names:
  getattr(mod,i).reset_index(drop=True,inplace=True)

#익월=금월+익월로 누적값 생성
for i in range(1,13):
  getattr(mod,'df'+'%d'%(i+1))['caseId']= getattr(mod,'df'+'%d'%i)['caseId']+ getattr(mod,'df'+'%d'%(i+1))['caseId']

#마지막 달의 그래프 내림차순 정렬을 위해 df1 인덱스 재지정
df13.sort_values(by = 'caseId', ascending = True,inplace=True)
df1=df1.reindex(index =df13.index.tolist())

#데이터프레임 병합
accum = pd.concat([df1,df2],axis=0)
for i in names[2:]:
  accum = pd.concat([accum,getattr(mod,i)])
case_accum=accum

In [64]:
fig = px.bar(case_accum, x='province', y="caseId", color='province',animation_frame="month",animation_group='province',range_y=[0,300])

fig.update_layout(title={'text': "<b>지역별 집단감염 건수 (누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='누적확진자수',tickfont_size=12))


fig.show()

## 5. 지역별 집단감염 확진자 수

---


####집단감염 확진자 수 (월별)

In [65]:
dfmon = df

#월별 데이터프레임 저장할 동적변수 생성
names = []
for i in range(0,13):
  names.append("df"+str(i+1))

time = ['2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12','2021-01','2021-02']

#데이터프레임 월별 분리 후 변수에 저장
for i,j in zip(names,time):
  globals()[i]=dfmon[dfmon['briefingdate'].str.contains(j)]

import sys , numpy as np
mod = sys.modules[__name__]

#지역별로 묶고 sum한 월별 데이터프레임을 list a에 저장
a=[]
for i in names:
  a.append(getattr(mod,i).groupby('province').sum())

#a에서 꺼내서 데이터프레임마다 month 컬럼 추가
for i,j in zip(a,time):
  i['month']=j

#데이터 프레임 병합
coromonth= pd.concat([a[0], a[1]],axis=0)
for i in a[2:]:
  coromonth = pd.concat([coromonth,i])

#월별 province 개수 동일하게 만들기 
coromonth=pd.pivot_table(coromonth,index='month',columns='province', values='confirmed', fill_value=0)
coromonth=coromonth.unstack().rename('confirmed').reset_index()

In [66]:
fig = px.bar(coromonth, x='province', y="confirmed", color='province',animation_frame="month",animation_group='province',range_y=[0,4000])

fig.update_layout(title={'text': "<b>지역별 집단감염 확진자 수(월별)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='확진자수',tickfont_size=12))

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 900

fig.show()

####집단감염 확진자 수 (누적)

In [67]:
#데이터프레임 월별 분리 후 변수에 저장
for i,j in zip(names,time):
  globals()[i]=coromonth[coromonth['month']==j]

#각 데이터 프레임 인덱스 reset
for i in names:
  getattr(mod,i).reset_index(drop=True,inplace=True)

#익월=금월+익월로 누적값 생성
for i in range(1,13):
  getattr(mod,'df'+'%d'%(i+1))['confirmed']= getattr(mod,'df'+'%d'%i)['confirmed']+ getattr(mod,'df'+'%d'%(i+1))['confirmed']

#마지막 달의 그래프 내림차순 정렬을 위해 df1 인덱스 재지정
df13.sort_values(by = 'confirmed', ascending = True,inplace=True)
df1=df1.reindex(index =df13.index.tolist())

#데이터프레임 병합
accum = pd.concat([df1,df2],axis=0)
for i in names[2:]:
  accum = pd.concat([accum,getattr(mod,i)])
coro_accum=accum

In [68]:
fig = px.bar(coro_accum, x='province', y="confirmed", color='province',animation_frame="month",animation_group='province',range_y=[0,10000])

fig.update_layout(title={'text': "<b>지역별 집단감염 확진자 수(누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='누적확진자수',tickfont_size=12))
fig.show()

##지역별 인구 대비 집단감염 분석

---

In [76]:
pop = pd.read_excel('202102_202102_주민등록인구및세대현황_월간.xlsx')
pop = pop.iloc[:,[1,2]]
pop = pop.rename(columns = pop.iloc[1])
pop.drop([0],axis = 0, inplace = True)
pop.columns = ['시도명', '인구수']
pop.reset_index(drop = True,inplace = True)
pop['시도명']=['서울','부산','대구','인천','광주','대전','울산','세종','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주도']

#각 케이스의 마지막 확진 데이터만 남김
case_df1=df.drop_duplicates('caseId',keep='last')
#지역별로 그룹화하여 count
case_df2=case_df1.groupby('province').count() 
case_df2=case_df2[['caseId']]
case_df2.drop(['etc.','해외유입'],axis=0,inplace = True)
case_df2.reset_index(inplace=True)
case_df2.columns = ['시도명', '누적사건수']
casepop=pd.merge(pop, case_df2, on='시도명', sort=False)
casepop['인구수대비사건수']=casepop['누적사건수']/casepop['인구수']*1000
casepop.sort_values(by = '인구수대비사건수',ascending = True,inplace = True)
casepop

,시도명,인구수,누적사건수,인구수대비사건수
6,울산,1132953,10,0.008826
13,전라남도,1849795,20,0.010812
15,경상남도,3336045,37,0.011091
12,전라북도,1801319,21,0.011658
5,대전,1462064,18,0.012311
16,제주도,675137,9,0.013331
3,인천,2941705,45,0.015297
10,충청북도,1599122,26,0.016259
11,충청남도,2119542,35,0.016513
7,세종,359569,6,0.016687


In [77]:
fig=px.bar(casepop, x = '시도명',y = '인구수대비사건수',hover_name="시도명",hover_data=["인구수대비사건수","인구수","누적사건수"],color_discrete_sequence=['#FFB000'])

fig.update_layout(title={'text': "<b>지역별 인구 대비 집단감염 건수(누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='인구대비사건수',tickfont_size=12))

In [78]:
df.sort_values(by = 'briefingdate', ascending = True,inplace = True)
#각 케이스의 마지막 확진 데이터만 남김
coro_df1=df.drop_duplicates('caseId',keep='last')
#지역별로 그룹화하여 각 열의 값을 sum
coro_df2 = coro_df1.groupby('province').sum()
coro_df2 = coro_df2[['accumulated']]
coro_df2.drop(['etc.','해외유입'],axis = 0,inplace = True)
coro_df2.reset_index(inplace=True)
coro_df2.columns = ['시도명', '누적확진자수']
coropop = pd.merge(pop, coro_df2, on='시도명', sort=False)
coropop['인구수대비확진자수'] = coropop['누적확진자수']/(coropop['인구수'])*1000
coropop.sort_values(by = '인구수대비확진자수',ascending = True,inplace = True)

In [79]:
fig=px.bar(coropop, x = '시도명',y = '인구수대비확진자수',hover_name="시도명",hover_data=["인구수대비확진자수","인구수","누적확진자수"],color_discrete_sequence=['#94E033'])

fig.update_layout(title={'text': "<b>지역별 인구 대비 집단감염 확진자 수(누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(title='',tickfont_size=12), yaxis=dict(title='인구대비확진자수(*1000)',tickfont_size=12))


##지역별 집단감염 사건 수와 환자 수

In [74]:
accum=pd.merge(coro_accum, case_accum,on=['province','month'], sort=False)
accum.rename(columns = {'confirmed' : '누적확진자수','caseId':'누적사건수'}, inplace = True)
accum

,province,month,누적확진자수,누적사건수
0,해외유입,2020-02,0,0
1,세종,2020-02,0,0
2,제주도,2020-02,0,0
3,전라남도,2020-02,0,0
4,울산,2020-02,0,0
...,...,...,...,...
242,광주,2021-02,1175,29
243,부산,2021-02,1300,65
244,경기도,2021-02,6048,241
245,대구,2021-02,6311,51


In [75]:
fig=px.scatter(accum, x="누적사건수", y="누적확진자수", animation_frame="month", animation_group="province",
           size="누적확진자수", color="province", hover_name="province", size_max=55, range_x=[0,300],range_y=[0,15000])

fig.update_layout(title={'text': "<b>지역별 집단감염 사건 수와 확진자 수(누적)<b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  title_font=dict(size=19,color='black'),
                  xaxis=dict(tickfont_size=12), yaxis=dict(tickfont_size=12))